In [ ]:
label_mapping = {
    'O': 0, 
    'B': 1,  
    'I': 2   
}
import json
import string
import nltk
import numpy as np
def clean_text(text):
    return text


def preprocess_data(data):
    texts = []
    labels = []
    for case_id, case_data in data.items():
        text = case_data['text'].lower()
        labels_for_case = case_data['labels']  
        
        # Tokenize the text into words
        words = nltk.word_tokenize(text)
        
        for i, word in enumerate(words):
            # Ensure that the index is within the range of labels_for_case
            if i < len(labels_for_case):
                # Find the label for this word
                label = labels_for_case[i]
                
                # Append the word and its corresponding label
                texts.append(word)
                labels.append(label_mapping[label])
            
    return texts, labels





# Load and preprocess train data
with open('/home/nalin21478/nlp_a2/task_2/dataset/train_bio.json', 'r') as f:
    train_data = json.load(f)
train_texts, train_labels = preprocess_data(train_data)

# Load and preprocess test data
with open('/home/nalin21478/nlp_a2/task_2/dataset/test_bio.json', 'r') as f:
    test_data = json.load(f)
test_texts, test_labels = preprocess_data(test_data)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

# Convert texts to sequences of word indices
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad sequences to 100
max_sequence_length = 100  
train_sequences_padded = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)



In [ ]:
from keras.utils import to_categorical
y_test = to_categorical(test_labels)

In [120]:
from collections import Counter

def convert_probabilities_to_tags(predictions, test_texts):
    tagged_texts = []
    for idx, seq_preds in enumerate(predictions):
        tags = []
        for prob in seq_preds:
            max_index = np.argmax(seq_preds)  # Get the index with the highest probability
            if max_index == 0:
                tags.append('O')
            elif max_index == 1:
                tags.append('B')
            elif max_index == 2:
                tags.append('I')
            else:
                # Handle unknown tag indices if present
                tags.append('UNKNOWN')
        
        # Select the most frequent tag for each word
        word_tags = dict(zip(test_texts[idx], tags))
        first_tag = list(word_tags.values())[0]
        tagged_texts.append((test_texts[idx], [first_tag] ))
    return tagged_texts

# Convert predictions to tagged texts for each model


def print_B_I_terms(tagged_texts):
    prev_B_term = None
    for text, tags in tagged_texts:
        for i in range(len(tags)):
            if tags[i] == 'B' and tags[i] != 'I':
                prev_B_term = text
                print("Aspect:", text)
            elif tags[i] == 'I':
                if prev_B_term:
                    print("Aspect:", prev_B_term+" "+text)
                
            else:
                prev_B_term = None


In [121]:
import pickle


with open("/home/nalin21478/nlp_a2/Task 2 Fasttext/Models/t2_LSTM_fasttext.pkl", "rb") as file:
    t2_LSTM_fasttext = pickle.load(file)


with open("/home/nalin21478/nlp_a2/Task 2 Fasttext/Models/t2_RNN_fasttext.pkl", "rb") as file:
    t2_RNN_fasttext = pickle.load(file)


with open("/home/nalin21478/nlp_a2/Task 2 Fasttext/Models/t2_GRU_fasttext.pkl", "rb") as file:
    t2_GRU_fasttext = pickle.load(file)

2024-03-11 03:56:39.266778: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 03:56:39.270247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 03:56:39.273494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [122]:
_, test_accuracy, test_f1_score = t2_RNN_fasttext.evaluate(test_sequences_padded, y_test)
print("RNN Fasttext ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_RNN_Fasttext= t2_RNN_fasttext.predict(test_sequences_padded)
tagged_texts_RNN_Fasttext=convert_probabilities_to_tags(Predictions_RNN_Fasttext, test_texts)
print_B_I_terms(tagged_texts_RNN_Fasttext)

_, test_accuracy, test_f1_score = t2_LSTM_fasttext.evaluate(test_sequences_padded, y_test)
print("LSTM Fasttext ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_LSTM_Fasttext= t2_LSTM_fasttext.predict(test_sequences_padded)
tagged_texts_LSTM_Fasttext=convert_probabilities_to_tags(Predictions_LSTM_Fasttext, test_texts)
print_B_I_terms(tagged_texts_LSTM_Fasttext)

_, test_accuracy, test_f1_score = t2_GRU_fasttext.evaluate(test_sequences_padded, y_test)
print("GRU Fasttext ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_GRU_Fasttext= t2_GRU_fasttext.predict(test_sequences_padded)
tagged_texts_GRU_Fasttext=convert_probabilities_to_tags(Predictions_GRU_Fasttext, test_texts)
print_B_I_terms(tagged_texts_GRU_Fasttext)

162/162 [==============================] - 6s 33ms/step - loss: 0.3070 - accuracy: 0.8936 - f1_score: 0.5612
RNN Fasttext 
Test Accuracy: 0.8936375975608826
Test F1 Score: 0.5611585974693298
162/162 [==============================] - 6s 38ms/step
Aspect: boot
Aspect: windows
Aspect: fan
Aspect: speakers
Aspect: price
Aspect: tag
Aspect: os
Aspect: features
Aspect: battery
Aspect: battery life
Aspect: key
Aspect: graphics
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: windows
Aspect: size
Aspect: speed
Aspect: screen
Aspect: price
Aspect: hardware
Aspect: keyboard
Aspect: windows
Aspect: setup
Aspect: windows
Aspect: usb
Aspect: usb ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: quality
Aspect: os
Aspect: ms
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: windows
Aspect: osx
Aspect: graphics
Aspect: apps
Aspect: speed
Aspect: touch
Aspect: windows
Aspect: size
Aspect: fan
Aspect: warranty
Aspect: windows
Aspect: bluetooth
Aspect: usb
Aspec

2024-03-11 03:57:02.320025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 03:57:02.322036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 03:57:02.323839: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

162/162 [==============================] - 14s 81ms/step - loss: 0.2957 - accuracy: 0.9006 - f1_score: 0.5961
LSTM Fasttext 
Test Accuracy: 0.900599479675293
Test F1 Score: 0.5961120128631592
  1/162 [..............................] - ETA: 1:17

2024-03-11 03:57:16.528456: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 03:57:16.530473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 03:57:16.531989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

162/162 [==============================] - 14s 84ms/step
Aspect: boot
Aspect: support
Aspect: windows
Aspect: fan
Aspect: speakers
Aspect: price
Aspect: os
Aspect: features
Aspect: battery
Aspect: battery life
Aspect: graphics
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: windows
Aspect: size
Aspect: speed
Aspect: screen
Aspect: price
Aspect: hardware
Aspect: keyboard
Aspect: windows
Aspect: setup
Aspect: windows
Aspect: usb
Aspect: ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: quality
Aspect: os
Aspect: ms
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: windows
Aspect: osx
Aspect: graphics
Aspect: built-in
Aspect: apps
Aspect: speed
Aspect: touch
Aspect: operating
Aspect: operating system
Aspect: windows
Aspect: size
Aspect: fan
Aspect: warranty
Aspect: support
Aspect: windows
Aspect: bluetooth
Aspect: usb
Aspect: firewire
Aspect: firewire port
Aspect: durability
Aspect: specs
Aspect: quality
Aspect: size
Aspect: windows
Aspect: window

2024-03-11 03:57:37.486925: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 03:57:37.489255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 03:57:37.491005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

162/162 [==============================] - 17s 101ms/step - loss: 0.2893 - accuracy: 0.9047 - f1_score: 0.6122
GRU Fasttext 
Test Accuracy: 0.9046605825424194
Test F1 Score: 0.6122375130653381


2024-03-11 03:57:54.912253: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 03:57:54.914135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 03:57:54.915513: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

162/162 [==============================] - 17s 104ms/step
Aspect: boot
Aspect: support
Aspect: windows
Aspect: functions
Aspect: fan
Aspect: internal
Aspect: speakers
Aspect: hard
Aspect: price
Aspect: os
Aspect: features
Aspect: battery
Aspect: battery life
Aspect: track
Aspect: track pad
Aspect: graphics
Aspect: build
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: windows
Aspect: size
Aspect: weight
Aspect: performance
Aspect: speed
Aspect: screen
Aspect: price
Aspect: hardware
Aspect: performance
Aspect: keyboard
Aspect: windows
Aspect: setup
Aspect: windows
Aspect: usb
Aspect: ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: performance
Aspect: quality
Aspect: performance
Aspect: os
Aspect: ms
Aspect: performance
Aspect: performance
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: hard
Aspect: windows
Aspect: osx
Aspect: graphics
Aspect: colors
Aspect: built-in
Aspect: apps
Aspect: speed
Aspect: touch
Aspect: gestures
Aspect: operating
A

In [ ]:
import pickle


with open("/home/nalin21478/nlp_a2/Task 2 Glove/Models/t2_RNN_Glove.pkl", "rb") as file:
    t2_RNN_Glove = pickle.load(file)


with open("/home/nalin21478/nlp_a2/Task 2 Glove/Models/t2_LSTM_Glove.pkl", "rb") as file:
    t2_LSTM_Glove = pickle.load(file)


with open("/home/nalin21478/nlp_a2/Task 2 Glove/Models/t2_GRU_Glove.pkl", "rb") as file:
    t2_GRU_Glove = pickle.load(file)

In [123]:
_, test_accuracy, test_f1_score = t2_RNN_Glove.evaluate(test_sequences_padded, y_test)
print("RNN Glove ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_RNN_Glove= t2_RNN_Glove.predict(test_sequences_padded)
tagged_texts_RNN_Glove=convert_probabilities_to_tags(Predictions_RNN_Glove, test_texts)
print_B_I_terms(tagged_texts_RNN_Glove)


_, test_accuracy, test_f1_score = t2_LSTM_Glove.evaluate(test_sequences_padded, y_test)
print("LSTM Glove ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_LSTM_Glove= t2_LSTM_Glove.predict(test_sequences_padded)
tagged_texts_LSTM_Glove=convert_probabilities_to_tags(Predictions_LSTM_Glove, test_texts)
print_B_I_terms(tagged_texts_LSTM_Glove)


_, test_accuracy, test_f1_score = t2_GRU_Glove.evaluate(test_sequences_padded, y_test)
print("GRU Glove ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_GRU_Glove= t2_GRU_Glove.predict(test_sequences_padded)
tagged_texts_GRU_Glove=convert_probabilities_to_tags(Predictions_GRU_Glove, test_texts)
print_B_I_terms(tagged_texts_GRU_Glove)



162/162 [==============================] - 5s 30ms/step - loss: 0.3026 - accuracy: 0.8992 - f1_score: 0.5598
RNN Glove 
Test Accuracy: 0.8992457985877991
Test F1 Score: 0.5598448514938354
162/162 [==============================] - 5s 30ms/step
Aspect: boot
Aspect: functions
Aspect: fan
Aspect: speakers
Aspect: price
Aspect: os
Aspect: features
Aspect: battery
Aspect: battery life
Aspect: graphics
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: size
Aspect: performance
Aspect: speed
Aspect: screen
Aspect: price
Aspect: performance
Aspect: keyboard
Aspect: usb
Aspect: ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: performance
Aspect: quality
Aspect: performance
Aspect: os
Aspect: portability
Aspect: ms
Aspect: performance
Aspect: performance
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: osx
Aspect: graphics
Aspect: colors
Aspect: built-in
Aspect: apps
Aspect: speed
Aspect: operating
Aspect: system
Aspect: size
Aspect: fan
Aspect: warranty


In [ ]:
import pickle


with open("/home/nalin21478/nlp_a2/Task 2 Word2Vec/Models/t2_RNN_word2vec.pkl", "rb") as file:
    t2_RNN_W2V = pickle.load(file)


with open("/home/nalin21478/nlp_a2/Task 2 Word2Vec/Models/t2_LSTM_word2vec.pkl", "rb") as file:
    t2_LSTM_W2V = pickle.load(file)


with open("/home/nalin21478/nlp_a2/Task 2 Word2Vec/Models/t2_GRU_word2vec.pkl", "rb") as file:
    t2_GRU_W2V = pickle.load(file)

In [124]:
_, test_accuracy, test_f1_score = t2_RNN_W2V.evaluate(test_sequences_padded, y_test)
print("RNN W2V ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_RNN_W2V= t2_RNN_W2V.predict(test_sequences_padded)
tagged_texts_RNN_W2V=convert_probabilities_to_tags(Predictions_RNN_W2V, test_texts)
print_B_I_terms(tagged_texts_RNN_W2V)


_, test_accuracy, test_f1_score = t2_LSTM_W2V.evaluate(test_sequences_padded, y_test)
print("LSTM W2V ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_LSTM_W2V= t2_LSTM_W2V.predict(test_sequences_padded)
tagged_texts_LSTM_W2V=convert_probabilities_to_tags(Predictions_LSTM_W2V, test_texts)
print_B_I_terms(tagged_texts_LSTM_W2V)


_, test_accuracy, test_f1_score = t2_GRU_W2V.evaluate(test_sequences_padded, y_test)
print("GRU W2V ")
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1_score)
Predictions_GRU_W2V= t2_GRU_W2V.predict(test_sequences_padded)
tagged_texts_GRU_W2V=convert_probabilities_to_tags(Predictions_GRU_W2V, test_texts)
print_B_I_terms(tagged_texts_GRU_W2V)



162/162 [==============================] - 5s 32ms/step - loss: 0.2970 - accuracy: 0.9056 - f1_score: 0.6106
RNN W2V 
Test Accuracy: 0.9056275486946106
Test F1 Score: 0.6106328964233398
162/162 [==============================] - 6s 35ms/step
Aspect: windows
Aspect: functions
Aspect: fan
Aspect: speakers
Aspect: price
Aspect: os
Aspect: features
Aspect: battery
Aspect: battery life
Aspect: graphics
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: windows
Aspect: size
Aspect: weight
Aspect: performance
Aspect: speed
Aspect: screen
Aspect: price
Aspect: hardware
Aspect: performance
Aspect: keyboard
Aspect: windows
Aspect: setup
Aspect: configure
Aspect: windows
Aspect: usb
Aspect: usb ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: performance
Aspect: quality
Aspect: performance
Aspect: os
Aspect: portability
Aspect: performance
Aspect: performance
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: microsoft
Aspect: windows
Aspect: microsoft
Aspec

2024-03-11 04:00:08.153638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 04:00:08.155823: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 04:00:08.157697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

162/162 [==============================] - 15s 90ms/step
Aspect: boot
Aspect: windows
Aspect: functions
Aspect: fan
Aspect: speakers
Aspect: hard
Aspect: price
Aspect: os
Aspect: features
Aspect: battery
Aspect: battery life
Aspect: graphics
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: windows
Aspect: size
Aspect: weight
Aspect: performance
Aspect: speed
Aspect: screen
Aspect: price
Aspect: hardware
Aspect: performance
Aspect: keyboard
Aspect: windows
Aspect: windows
Aspect: usb
Aspect: usb ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: performance
Aspect: quality
Aspect: performance
Aspect: os
Aspect: portability
Aspect: performance
Aspect: performance
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: hard
Aspect: windows
Aspect: graphics
Aspect: colors
Aspect: speed
Aspect: gestures
Aspect: operating
Aspect: operating system
Aspect: windows
Aspect: size
Aspect: fan
Aspect: warranty
Aspect: windows
Aspect: aluminum
Aspect: bluetooth
Aspe

2024-03-11 04:00:43.969473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-11 04:00:43.972274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-11 04:00:43.974165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

162/162 [==============================] - 17s 101ms/step
Aspect: boot
Aspect: windows
Aspect: functions
Aspect: fan
Aspect: speakers
Aspect: hard
Aspect: price
Aspect: os
Aspect: features
Aspect: wifi
Aspect: connection
Aspect: battery
Aspect: battery life
Aspect: graphics
Aspect: build
Aspect: durability
Aspect: battery
Aspect: battery life
Aspect: windows
Aspect: size
Aspect: weight
Aspect: performance
Aspect: speed
Aspect: screen
Aspect: price
Aspect: hardware
Aspect: performance
Aspect: keyboard
Aspect: windows
Aspect: setup
Aspect: windows
Aspect: usb
Aspect: usb ports
Aspect: features
Aspect: screen
Aspect: keyboard
Aspect: performance
Aspect: quality
Aspect: performance
Aspect: os
Aspect: portability
Aspect: ms
Aspect: performance
Aspect: look
Aspect: performance
Aspect: keys
Aspect: screen
Aspect: display
Aspect: os
Aspect: hard
Aspect: microsoft
Aspect: windows
Aspect: osx
Aspect: microsoft
Aspect: graphics
Aspect: colors
Aspect: built-in
Aspect: speed
Aspect: touch
Aspect: o